In [ ]:
import numpy as np
from numpy import unravel_index
import math
import matplotlib.pyplot as plt
from IPython import display
from numpy import linalg as LA
import cmath


# Method to get acceleration for each particle
def getAcc(ps, m, l, length):
    
    num1 = len(ps);
    a = np.zeros((num1,3));
    
    #Newtonian force of gravity calculated for all non-mirror black holes
    for i in range(num1 - length):
        
        for k in range(num1):
            
            for j in range(3):
                
                if (k != i and ((k != len(ps) - length + i) and (i != len(ps) - length + k))):
                    
                    a[i][j] += m[k]*(ps[k][j] - ps[i][j])*((ps[i][1] - ps[k][1])**2 + (ps[i][2] - ps[k][2])**2 + (ps[i][0] - ps[k][0])**2)**(-3/2)
    
    #Newtonian force of gravity calculated with mirror black holes' effect
    for i in range(length):   
        
        imr = len(ps) - length + i;
        
        for k in range(num1):
            
            for j in range(3):
                
                if (k != i and k != imr):
                    
                    a[i][j] += m[k]*(ps[k][j] - ps[imr][j])*((ps[imr][1] - ps[k][1])**2 + (ps[imr][2] - ps[k][2])**2 + (ps[imr][0] - ps[k][0])**2)**(-3/2)
    
    #Acceleration of boundary black hole and its mirror is equal
    for i in range(length):
        
        a[num1 - length + i] = a[i];
    
    return a

#Method to enforce a 3D Torus Metric
def torus(ps, l, Ncr, length, acount):
    
    Ncr -= 1;
    l = l*(2**(acount));
    
    #Checking if a boundary/mirror black hole goes past the ends of our toroidal lattice
    for i in range(length):
        
        #Categorizing mirror and boundary black hole pairs based on which direction they are in relation to each other
        btype = np.zeros((3,1));
        
        for k in range(3):
            
            if (ps[i][k] - ps[len(ps) - length + i][k] > 1):
                
                btype[k] = 1;
        
        for j in range(3):
            
            if (btype[j] == 1):
            
                if ((ps[i][j] < -l*Ncr/2 - 1 and ps[len(ps) - length + i][j] < l*Ncr/2 - 1) or (ps[i][j] < l*Ncr/2 - 1 and ps[len(ps) - length + i][j] < -l*Ncr/2 - 1)):
                    
                    ps[i][j] = ps[i][j] + l*Ncr;
                    ps[len(ps) - length + i][j] = ps[len(ps) - length + i][j] + l*Ncr;
                    
                if ((ps[i][j] > l*Ncr/2 + 1 and ps[len(ps) - length + i][j] > 3*l*Ncr/2 + 1) or (ps[i][j] > 3*l*Ncr/2 + 1 and ps[len(ps) - length + i][j] > l*Ncr/2 + 1)):
                
                    ps[i][j] = ps[i][j] - l*Ncr;
                    ps[len(ps) - length + i][j] = ps[len(ps) - length + i][j] - l*Ncr;
                    
            else:
                
                if ((ps[i][j] < -l*Ncr - 1 and ps[len(ps) - length + i][j] < -2*l*Ncr - 1) or (ps[i][j] < -2*l*Ncr - 1 and ps[len(ps) - length + i][j] < -l*Ncr - 1)):

                    ps[i][j] = ps[i][j] + l*Ncr;
                    ps[len(ps) - length + i][j] = ps[len(ps) - length + i][j] + l*Ncr;
                    
                if ((ps[i][j] > l*Ncr + 1 and ps[len(ps) - length + i][j] > 2*l*Ncr + 1) or (ps[i][j] > 2*l*Ncr + 1 and ps[len(ps) - length + i][j] > l*Ncr + 1)):
                
                    ps[i][j] = ps[i][j] - l*Ncr;
                    ps[len(ps) - length + i][j] = ps[len(ps) - length + i][j] - l*Ncr;
    
    #Checking if interior black holes go past the ends of our toroidal lattice
    for i in range(length, len(ps) - length):
        
        for j in range(3):
            
            if (ps[i][j] < -l*Ncr - 1):

                    ps[i][j] = ps[i][j] + l*Ncr;
                    
            if (ps[i][j] > l*Ncr + 1):
                
                    ps[i][j] = ps[i][j] - l*Ncr;
    
    return ps;
    
#merge method
def merge(ps, m, vl, acl, length, l, N):
    
    #Uses output of the distance array
    output = distance(ps, N, m);
    dist = output[0];
    boolean = output[1];
    
    #Merges all black holes close to each other
    while max(boolean) != 0:
        
        #Finding which pair should be merged
        indarray = [];
        mindistarray = [];
        
        for i in range(len(boolean)):
            
            if (boolean[i][0] > 0):
                
                indarray.append(i);
                mindistarray.append(min(dist[i]));
                        
        coord = np.argwhere(dist == np.min(mindistarray));
    
        i = coord[0][0];
        j = coord[0][1];

        #Actually merging the black holes based on position
        if (dist[i][j] <= 10*m[i] or dist[i][j] <= 10*m[j]):
            
            N = N - 1;
            
            #Two boundary
            if (i < length and j < length):
                
                vl[j] = (m[j]*vl[j] + m[i]*vl[i])/(m[i] + m[j]);
                
                m[j] = m[i] + m[j];
                ps[j] = (ps[i] + ps[j])/2;
                
                imr = len(ps) - length + i;
                jmr = len(ps) - length + j;
                
                vl[jmr] = vl[j];
                
                m[jmr] = m[imr] + m[jmr];
                ps[jmr] = (ps[imr] + ps[jmr])/2;
                
                m = np.delete(m,i,0); 
                ps = np.delete(ps,i,0);
                dist = np.delete(dist,i,0);
                dist = np.delete(dist,i,1);
                
                m = np.delete(m,imr - 1,0); 
                ps = np.delete(ps, imr - 1,0);
                dist = np.delete(dist,imr - 1,0);
                dist = np.delete(dist,imr - 1,1);
                
                vl = np.delete(vl,i,0);
                acl = np.delete(acl,i,0);
                
                vl = np.delete(vl,imr - 1,0);
                acl = np.delete(acl,imr - 1,0);

                length -= 1;
                N = N - 1;
                
            #Two mirrors
            elif (i > (len(ps) - length - 1) and j > (len(ps) - length - 1)):
                
                vl[j] = (m[j]*vl[j] + m[i]*vl[i])/(m[i] + m[j]);
                
                m[j] = m[i] + m[j];
                ps[j] = (ps[i] + ps[j])/2;
                
                imr = -len(ps) + length + i;
                jmr = -len(ps) + length + j;
                
                m[jmr] = m[imr] + m[jmr];
                ps[jmr] = (ps[imr] + ps[jmr])/2;
                
                vl[jmr] = vl[j];
                
                m = np.delete(m,i,0); 
                ps = np.delete(ps,i,0);
                dist = np.delete(dist,i,0);
                dist = np.delete(dist,i,1);
                
                m = np.delete(m,imr - 1,0); 
                ps = np.delete(ps, imr - 1,0);
                dist = np.delete(dist,imr - 1,0);
                dist = np.delete(dist,imr - 1,1);

                vl = np.delete(vl,i,0);
                acl = np.delete(acl,i,0);
                
                vl = np.delete(vl,imr - 1,0);
                acl = np.delete(acl,imr - 1,0);

                length -= 1;
                N = N - 1;
                
            #One boundary and one interior
            elif (((i < length or i > len(ps) - length - 1) ^ (j < length or j > len(ps) - length - 1)) and ((j != len(ps) - length + i) and (i != len(ps) - length + j))):
                
                if (i < length or i > len(ps) - length - 1):
                    
                    if (i < length):
                        
                        vl[j] = (m[j]*vl[j] + m[i]*vl[i])/(m[i] + m[j]);
                        
                        m[j] = m[i] + m[j];
                        ps[j] = (ps[i] + ps[j])/2;

                        imr = len(ps) - length + i;

                        m = np.delete(m,i,0); 
                        ps = np.delete(ps,i,0);
                        dist = np.delete(dist,i,0);
                        dist = np.delete(dist,i,1);
                        
                        m = np.delete(m,imr-1,0); 
                        ps = np.delete(ps,imr-1,0);
                        dist = np.delete(dist,imr-1,0);
                        dist = np.delete(dist,imr-1,1);

                        vl = np.delete(vl,i,0);
                        acl = np.delete(acl,i,0);
                        
                        vl = np.delete(vl,imr-1,0);
                        acl = np.delete(acl,imr-1,0);

                        length -= 1;
                        N -= 1;
                
                        
                    else:
                        
                        vl[j] = (m[j]*vl[j] + m[i]*vl[i])/(m[i] + m[j]);
                        
                        m[j] = m[i] + m[j];
                        ps[j] = (ps[i] + ps[j])/2;

                        iorg = -len(ps) + length + i;

                        m = np.delete(m,i,0); 
                        ps = np.delete(ps,i,0);
                        dist = np.delete(dist,i,0);
                        dist = np.delete(dist,i,1);
                        
                        m = np.delete(m,iorg,0); 
                        ps = np.delete(ps,iorg,0);
                        dist = np.delete(dist,iorg,0);
                        dist = np.delete(dist,iorg,1);

                        vl = np.delete(vl,i,0);
                        acl = np.delete(acl,i,0);
                        
                        vl = np.delete(vl,iorg,0);
                        acl = np.delete(acl,iorg,0);

                        length -= 1;
                        N -= 1;
                        
                else:
                    
                    if (j < length):
                        
                        vl[i] = (m[j]*vl[j] + m[i]*vl[i])/(m[i] + m[j]);
                        
                        m[i] = m[i] + m[j];
                        ps[i] = (ps[i] + ps[j])/2;

                        jmr = len(ps) - length + j;

                        m = np.delete(m,j,0); 
                        ps = np.delete(ps,j,0);
                        dist = np.delete(dist,j,0);
                        dist = np.delete(dist,j,1);
                        
                        m = np.delete(m,jmr-1,0); 
                        ps = np.delete(ps,jmr-1,0);
                        dist = np.delete(dist,jmr-1,0);
                        dist = np.delete(dist,jmr-1,1);

                        vl = np.delete(vl,j,0);
                        acl = np.delete(acl,j,0);
                        
                        vl = np.delete(vl,jmr-1,0);
                        acl = np.delete(acl,jmr-1,0);

                        length -= 1;
                        N -= 1;
                        
                    else:
                        
                        vl[i] = (m[j]*vl[j] + m[i]*vl[i])/(m[i] + m[j]);
                        
                        m[i] = m[i] + m[j];
                        ps[i] = (ps[i] + ps[j])/2;

                        jorg = -len(ps) + length + i;

                        m = np.delete(m,j,0); 
                        ps = np.delete(ps,j,0);
                        dist = np.delete(dist,j,0);
                        dist = np.delete(dist,j,1);

                        vl = np.delete(vl,j,0);
                        acl = np.delete(acl,j,0);
                        
                        m = np.delete(m,jorg,0); 
                        ps = np.delete(ps,jorg,0);
                        dist = np.delete(dist,jorg,0);
                        dist = np.delete(dist,jorg,1);

                        vl = np.delete(vl,jorg,0);
                        acl = np.delete(acl,jorg,0);

                        length -= 1;
                        N -= 1;
                
            #Boundary + Mirror
            elif (((i < length or i > len(ps) - length - 1) and (j < length or j > len(ps) - length - 1)) and ((j != len(ps) - length + i) and (i != len(ps) - length + j))):
                
                if (i < length and j > len(ps) - length - 1):

                        
                    imr = len(ps) - length + i;
                    jorg = -1*len(ps) + length + j;  
                    
                    vl[i] = (m[j]*vl[j] + m[i]*vl[i])/(m[i] + m[j]);
                    
                    m[i] = m[i] + m[j];
                    deltaps = (-ps[i] + ps[j])/2; 
                    ps[i] = (ps[i] + ps[j])/2; 

                    m[imr] = m[i];
                    ps[imr] += deltaps;
                    
                    vl[imr] = vl[i];

                    m = np.delete(m,jorg,0); 
                    ps = np.delete(ps,jorg,0);
                    dist = np.delete(dist,jorg,0);
                    dist = np.delete(dist,jorg,1);

                    m = np.delete(m,j - 1,0); 
                    ps = np.delete(ps, j - 1,0);
                    dist = np.delete(dist,j - 1,0);
                    dist = np.delete(dist,j - 1,1);

                    vl = np.delete(vl,jorg,0);
                    acl = np.delete(acl,jorg,0);

                    vl = np.delete(vl,j - 1,0);
                    acl = np.delete(acl,j - 1,0);

                    length -= 1;
                    N = N - 1;
                        
                else:
                    
                    vl[i] = (m[j]*vl[j] + m[i]*vl[i])/(m[i] + m[j]);

                    jmr = len(ps) - length + j;
                    iorg = -1*len(ps) + length + i;

                    m[i] = m[i] + m[j];
                    deltaps = (-ps[i] + ps[j])/2; 
                    ps[i] = (ps[i] + ps[j])/2;

                    m[iorg] = m[i];
                    ps[iorg] += deltaps;
                    
                    vl[iorg] = vl[i];
                    
                    m = np.delete(m,j,0); 
                    ps = np.delete(ps,j,0);
                    dist = np.delete(dist,j,0);
                    dist = np.delete(dist,j,1);

                    m = np.delete(m,jmr - 1,0); 
                    ps = np.delete(ps, jmr - 1,0);
                    dist = np.delete(dist,jmr - 1,0);
                    dist = np.delete(dist,jmr - 1,1);

                    vl = np.delete(vl,j,0);
                    acl = np.delete(acl,j,0);

                    vl = np.delete(vl,jmr - 1,0);
                    acl = np.delete(acl,jmr - 1,0);

                    length -= 1;
                    N = N - 1;
            
            #Two interiors
            else:
                
                vl[j] = (m[j]*vl[j] + m[i]*vl[i])/(m[i] + m[j]);
                
                m[j] = m[i] + m[j];
                ps[j] = (ps[i] + ps[j])/2;

                m = np.delete(m,i,0); 
                ps = np.delete(ps,i,0);
                dist = np.delete(dist,i,0);
                dist = np.delete(dist,i,1);

                vl = np.delete(vl,i,0);
                acl = np.delete(acl,i,0);
                

        output = distance(ps, N, m);
        dist = output[0];
        boolean = output[1];

    return m, ps, vl, acl, N, length;

#Get distance between particles and check which ones are close enough to merge
def distance(pos, N, m):
    
    dist = np.zeros((N,N));
    boolean = np.zeros((N,1));
    sr = 10*m;
    
    for i in range(N):
        for j in range(N):
                dist[i][j] = ((pos[i][1] - pos[j][1])**2 + (pos[i][2] - pos[j][2])**2 + (pos[i][0] - pos[j][0])**2)**(0.5);

    for i in range(N):
        dist[i][i] = 10**20;
        
    for i in range(N):
        if (np.amin(dist[i]) <= 10*m[i]):
            
            boolean[i] = 1;
    
    return dist, boolean;

#Method to help debug
def checkdist(x,y):
    
    dist = ((x[1] - y[1])**2 + (x[2] - y[2])**2 + (x[0] - y[0])**2)**(0.5);
    
    print(dist);

def main(const, vsd, msd, N):
    
    #Necessary variables
    Ncr = int((N)**(1/3));
    t = 0
    tf = 2**(10)*math.pi*10**(16);
    dt = 5*tf/10**14;
    softening = 1
    plotTime = True 
    
    #physical variables
    np.random.seed();
    lengthofps = int((Ncr - 2)**3);
    ps = np.zeros((int((N - lengthofps)/2),3));
    psm = np.zeros((len(ps),3));
    psi = np.zeros((lengthofps,3));
    
    stat = len(ps);
    
    m0 = 10**(6);
    mi = msd * np.random.randn(len(psi),1) + m0;
    m = msd * np.random.randn(len(ps),1) + m0;
    
    for i in range(len(m)):
        
        while (m[i] < 0):
            
            m[i] = np.random.normal(loc = m0,scale = msd);
            
    for i in range(len(mi)):
        
        while (mi[i] < 0):
            
            mi[i] = np.random.normal(loc = m0,scale = msd);
    
    mm = m;
    
    sr0 = 10*m0;
    
    mall = np.vstack((m,mi));
    mall = np.vstack((mall,mm));
    
    rhs = const*(m0**(-3))*(10**(-21));
    l = (N/rhs)**(1/3);
    c = 1;
    a = 1;
    
    #Setting up the initial toroidal lattice
    random2 = 0;

    for j in range(int(N**(1/3))):
        for k in range(int(N**(1/3))):
            if (j != 0  and j != N**(1/3)-1 and k != N**(1/3)-1 and k != 0):
                ps[random2][0] = 0
                ps[random2][1] = l*j
                ps[random2][2] = l*k
                random2 += 1;
            
    for j in range(int(N**(1/3))):
        for k in range(int(N**(1/3))):
            if ((j != 0) and j != N**(1/3)-1 and k != N**(1/3)-1 and k != 0):
                ps[random2][0] = l*j
                ps[random2][1] = 0
                ps[random2][2] = l*k
                random2 += 1;
            
    for j in range(int(N**(1/3))):
        for k in range(int(N**(1/3))):
            if (j != 0 and k != 0 and (j != N**(1/3)-1) and (k != N**(1/3)-1)):
                ps[random2][0] = l*j
                ps[random2][1] = l*k
                ps[random2][2] = 0
                random2 += 1;
                
    save = random2;
                
    ps[random2][0] = 0;
    ps[random2][1] = 0;
    ps[random2][2] = 0;
    
    random2+=1;
    
    ps[random2][0] = 0;
    ps[random2][1] = l*(Ncr-1);
    ps[random2][2] = 0;
    
    random2+=1;
    
    ps[random2][0] = l*(Ncr-1);
    ps[random2][1] = 0;
    ps[random2][2] = 0;
    
    random2+=1;
    
    ps[random2][0] = l*(Ncr-1);
    ps[random2][1] = l*(Ncr-1);
    ps[random2][2] = 0;
    
    random2+=1;
    
    
    for i in range(4):
        
        for j in range(Ncr - 2):
            
            if (i % 2 == 0):
                
                ps[random2][0] = l*(j + 1);
                ps[random2][1] = i*0.5*(Ncr-1)*l;
                ps[random2][2] = 0;
                random2+=1;
                
            else:
            
                ps[random2][0] = (i - 1)*0.5*(Ncr-1)*l;
                ps[random2][1] = l*(j + 1)
                ps[random2][2] = 0;
                random2+=1;
            
            
    for j in range(Ncr - 2):
        
        ps[random2][0] = 0
        ps[random2][1] = 0;
        ps[random2][2] = l*(j + 1)
        random2+=1;
        
    for j in range(Ncr - 2):
        
        ps[random2][0] = l*(Ncr-1);
        ps[random2][1] = 0;
        ps[random2][2] = l*(j + 1)
        random2+=1;
    
    #Creating mirrors for boundary black holes     
    for i in range(save):
        
        zindex = np.where(ps[i] == 0)[0];

        if zindex == 0:
            
            psm[i] = [l*(Ncr-1), ps[i][1], ps[i][2]];
            
        if zindex == 1:
            
            psm[i] = [ps[i][0], l*(Ncr-1), ps[i][2]];
            
        if zindex == 2:
            
            psm[i] = [ps[i][0], ps[i][1], l*(Ncr-1)];
    
    psm[save][0] = l*(Ncr - 1);
    psm[save][1] = l*(Ncr - 1);
    psm[save][2] = l*(Ncr - 1);
    
    save+=1;
    
    psm[save][0] = l*(Ncr - 1);
    psm[save][1] = 0;
    psm[save][2] = l*(Ncr - 1);
    
    save+=1;
    
    psm[save][0] = 0;
    psm[save][1] = l*(Ncr - 1);
    psm[save][2] = l*(Ncr - 1);
    
    save+=1;
    
    psm[save][0] = 0;
    psm[save][1] = 0;
    psm[save][2] = l*(Ncr - 1);
    
    save+=1;
    
    for i in range(4):
        
        for j in range(Ncr - 2):
            
            if (i % 2 == 0):
                
                psm[save][0] = l*(Ncr - 1 - (j + 1));
                psm[save][1] = l*(Ncr - 1) - i*0.5*(Ncr-1)*l;
                psm[save][2] = l*(Ncr - 1);
                save+=1;
                
            else:
            
                psm[save][0] = l*(Ncr - 1) - (i - 1)*0.5*(Ncr-1)*l;
                psm[save][1] = l*(Ncr - 1 -(j + 1));
                psm[save][2] = l*(Ncr-1);
                save+=1;
                
    for j in range(Ncr - 2):
        
        psm[save][0] = l*(Ncr - 1);
        psm[save][1] = l*(Ncr - 1);
        psm[save][2] = l*(Ncr - 1 - (j + 1));
        save+=1;
        
    for j in range(Ncr - 2):
        
        psm[save][0] = 0;
        psm[save][1] = l*(Ncr - 1);
        psm[save][2] = l*(Ncr - 1 - (j + 1));
        save+=1;
    
    random3 = 0;
    for i in range(1,Ncr - 1):
        for j in range(1,Ncr - 1):
            for k in range(1,Ncr - 1):
                psi[random3][0] = l*i;
                psi[random3][1] = l*j;
                psi[random3][2] = l*k;
                random3 += 1;
                
    psall = np.vstack((ps,psi));
    psall = np.vstack((psall,psm));
    
    #More fundamental variables
    vl = np.random.randn(len(ps),3)*vsd;
    
    acall = getAcc(psall, mall, l, len(ps));
        
    vlm = vl;  
    vli = np.random.randn(len(psi),3)*vsd;
    vlall = np.vstack((vl,vli));
    vlall = np.vstack((vlall,vlm));
    
    boundl = len(ps);
    
    Nt = int(np.ceil(tf/dt))
    
    acount = 0;
        
    length = len(ps);
    
    tt = 10**(21/2);
    
    for i in range(Nt):

        #Leapfrog Integration
        psall += vlall*dt + 0.5*acall*dt**2
        vlall += acall*dt/2.0

        acall = getAcc(psall, mall, l, length);
        vlall += acall*dt/2.0;

        psall = torus(psall,l, Ncr, length, acount); 

        result = merge(psall, mall, vlall, acall, length, l,N);

        mall = result[0];
        psall = result[1];
        vlall = result[2];
        acall = result[3];
        N = result[4];
        length = result[5];        
        
        #Scale factor a is introduced
        a = (tt/10**(21/2))**(2/3);
        
        #Expansion of the universe (stops at 10 times)
        if (a >= 2 and acount <= 10):

            psall = 2*psall;
            acount += 1;

            a = 1;

            tt = 10**(21/2);

        t += dt;
        tt += dt;

        if (acount < 10):
            
            print("A value is " + str(a));

    return (len(mall) - stat)/(N - stat);


if __name__== "__main__":
    
    #Number of times to run the simulation
    Nu = 1;
    
    data = np.zeros((Nu,1));

    N = 5**3;
    vsd = 0.25;
    const = 10**(18);
    msd = (10**6)/2;
    
    for i in range(Nu):
        
        print(i);
        stat = main(const, vsd, msd, N);
        print(stat);
        data[i] = stat;
        
    print(data);